# 9장 클라이언트 탐지
## 9.1 기능 탐지
* 클라이언트 탐지 중에서 가장 널리 쓰이는 방법
* 어떤 브라우저를 사용하는지 x
* 어떤 기능이 지원되는지 o

``` javascript
// 기능탐지의 기본 패턴
if (object.propertyInQuestion) {
    // object.propertyInQuestion 사용
}
```

* 예를 들어 DOM 메서드 document.getElementById() 는 IE5 이전버전에서 지원 하지 않음
* 같은 기능을 비표준 document.all 프로퍼티로 구현 가능

``` javascript
function getElementId(id) {
    if (document.getElementId) {
        return document.getElementId(id);
    } else if (document.all) {
        return document.all[id];
    } else {
        throw new Error('No way to retrieve element');
    }
}
```
* 표준방식인 document.getElementId가 존재하는 지 체크
* document.all 체크

---
* 기능 탐지의 중요한 두가지 개념
    * 가장 일반적인 방법을 제일 먼저 테스트해야한다
        * 위의 예제에서 document.getElementId 메서드를 먼저 체크함
        * 테스트 할 조건 수가 줄어들어 코드 실행이 능률적임
    * 사용하려는 기능을 정확히 체크
        * A 기능이 브라우저에 존재한다고 해서, B 기능이 존재하는 것은 아님
        
        ``` javascript
        function getWindowWidth() {
            if (document.all) {
                return document.documentElement.clientWidth; // 잘못된 사용
            } else {
                return window.innerWidth;
            }
        }
        ```

### 9.1.1 안전한 기능 탐지
* 기능탐지는 원하는 기능이 존재하는지만이 아니라 기능이 정확히 동작함을 확인 할 수 있을 때 가장 효과적
* 이전 예제에서 객체에 해당 멤버가 존재하는지는 알 수 있지만 그 멤버가 정확이 원하는 것이라고 확신 할 수는 없다.

In [ ]:
// 단순히 존재 여부만 확인하는 잘못된 방법
function isSortable(object) {
    return !!object.sort;
}

* 이 함수는 sort() 메서드가 있는지 체크해서 객체가 정렬 가능한지 확인
* 객체에 sort라는 프로퍼티가 있어도 true 반환 - 문제

In [ ]:
var result = isSortable({sort: true});
console.log(result);

* 단순히 프로퍼티의 존재 여부만 테스트해서는 객체가 정렬 가능한지 확실할 수 없다.
* 더 나은 방법은 sort가 정말 함수 인지 체크

In [ ]:
function isSortable(object) {
    return typeof object.sort == 'function';
}

* 가능한 한 기능 탐지에 typeof 를 써야 하지만 만능은 아님
* 특정 상황에서는 typeof 의 값을 정확히 반환한다고 확신하기 어려움
    * IE8 이전 버전에서 typeof document.createElement 는 object를 반환함
    
### 9.1.2 기능 탐지는 브라우저 탐지가 아닙니다.
* 특정 기능이나 기능 집합을 탐지할 때, 어떤 브라우저에서 실행 중인지 알 필요는 없다.
* 다음의 소위 '브라우저 탐지' 코드는 수많은 웹사이트에서 쓰이지만 기능 탐지를 잘못 사용한 사례

``` javascript
// 이렇게 하지 마십시오. 충분히 명시적이지 못합니다.
var isFirefox = !!(navigator.vendor && navigator.vendorSub);

// 이렇게 하지 마십시오. 너무 많은 걸 가정합니다.
var isIE = !!(document.all && document.uniqueID);
```

* 잘못된 기능 탐지의 고전적 사례
* 과거에는 navigator.vendor와 navigator.vendorSub만 체크 하면 파이어폭스임을 알 수 있었지만, 사파리에서 같은 프로퍼티를 구현하면서 부정확한 결과를 얻게 되었다.
* document.all, document.uniqueID 두 프로퍼티가 IE의 미래 버전에도 존재할 것이며, 다른 브라우저는 결코 구현하지 않을 것이라고 가정하는 것.
* 하지만 몇 가지 기능을 묶어서 브라우저 그룹을 만드는 방법은 적절

``` javascript
// 브라우저에서 넷스케이프 스타일 플러그인을 지원하는지 체크
var hasNSPlugins = !!(navigator.plugins && navigator.plugins.length);

// 브라우저에서 기본적인 DOM level 1 기능을 지원하는지 체크
var hasDOM1 = !!(document.getElementById && document.createElement && document.getElementsByTagName);
```

> note: 기능 탐지는 해결책을 찾지 못했을 때 쓰는 보험 같은 것이지, 어떤 브라우저에서 실행 중인지 알기 위함이 아니다.

## 9.2 쿽스 탐지
* 기능 탐지와 비슷한 개념으로 '쿽스 탐지'가 있다.
* 브라우저의 특정 동작방식을 찾아 내려는 것
* 지원되는 것을 찾는 대신 뭔가 정확히 동작하지 않는 것('쿽스'는 사실 '버그')을 찾아 내려 한다.
* 예를 들어 IE8 이전 버전에는 [[Enumerable]] 속성이 false로 지정된 인스턴스 프로퍼티가 있다면, 같은 이름의 프로토타입 프로퍼티를 for-in 루프에서 표시하지 않는 버그가 있음

``` javascript
var hasDontEnumQuirk = function() {
    var o = {toString: function() {} };
    for (var prop in o) {
        if (prop == "toString") {
            return false;
        }
    }
    
    return true;
}();
```

* 익명 함수를 이용해 쿽스 테스트
* toString() 메서드를 정의한 객체 생성
* 올바른 ECMAScript 구현에서는 for-in 루프에서 toString을 프로퍼티로 반환

---

* 이런 버그들은 보통 한 브라우저에 국한되며, 다음 버전에서 수정될 수도 있고 아닐 수도 있다.
* 쿽스 탐지는 코드를 실행해야 하므로 직접적인 영향이 있는 버그만 테스트하고, 가능한 스크립트 첫 부분에서 테스트해서 이를 배제하는 편이 좋다.

## 9.3 브라우저 탐지
* 가장 논란이 많은 클라이언트 탐지 방법
* 브라우저의 사용자 에이전트 문자열을 통해 어떤 브라우저에서 실행 중인지 확인
* 사용자 에이전트 문자열은 HTTP요청을 보낼 때마다 받는 응답헤더에 포함되어 있으며
* navigator.userAgent를 통해 접근
* 서버에서는 사용자 에이전트 문자열을 보고 어떤 브라우저를 사용 중인지 확인하고 그에 맞게 반응하도록 프로그램하는 일이 일반적이며 문제도 없다.
* 하지만 클라이언트에서는 기능탐지나 쿽스 탐지로 해결되지 않을 때, 최후의 수단으로 받아들여짐
* 사용자 에이전트 문자열에서 가장 심한 논란거리는 브라우저가 사용자 에이전트 문자열에 잘못된 정보를 넣어서 서버를 속이는 '위장'의 역사.

### 9.3.1 역사
* HTTP 명세는 브라우저는 브라우저 이름과 버전을 명시하는 짧은 사용자 에이전트 문자열을 전송해야 한다고 정함
* 추가적으로 토큰/제품 버전의 목록 형태로 만들기를 요구
* 하지만 현실에서 사용자 에이전트 문자열은 결코 그리 단순하지 않음

#### 초기 브라우저
* 첫 번째 웹 브라우저인 모자이크
    * 형식
        * Mosaic/0.9
* 넷스케이프 내비게이터 2
    * 코드네임 모질라
    * 형식
        * Mozilla/Version [Language] (Platform; Encryption)
    * 제품 이름과 버전을 문자열 맨 앞에 두고, 다음 정보를 추가함
        * 언어 - 애플리케이션에서 의도한 언어 코드
        * 플랫폼 - 애플리케이션을 실행하는 운영체제/플랫폼
        * Encryption - 보안에 사용된 암호화 타입
            * U(128비트 암호화)
            * I(40비트 암호화)
            * N(암호화 없음)
    * 일반적인 사용자 에이전트 문자열
        * Mozilla/2.02 [fr] (winNT; I)

#### 넷스케이프 내비게이터 3와 IE3
* 넷스케이프 내비게이터 3
    * 언어토큰을 제거하고 CPU나 운영체제에 대한 정보 추가
    * 형식
        * Mozilla/Version (Platform; Encryption[;OS-or-CPU description])
    * 윈도 시스템에서 실행되는 넷스케이프 내비게이터 3의 사용자 에이전트 문자열
        * Mozilla/3.0 (Win95; U)
* 마이크로소프트 IE3
    * 당시에는 넷스케이프 브라우저가 시장을 선점, 많은 서버에서 내비게이터인지 체크.
    * IE에서 브라우저를 판단할 수 없어서 페이지가 깨져 보임.
    * 넷스케이프 사용자 에이전트 문자열과 호환되게 만들기로 결정
    * 형식
        * Mozilla/2.0 (compatible; MSIE Version; Operating System)
    * 윈도 95에서 동작하는 IE3.02의 사용자 에이전트 문자열
        * Mozilla/2.0 (compatible; MSIE 3.02; Windows 95)
    * 당시의 브라우저 탐지 프로그램은 에이전트 문자열에서 제품 이름 부분만 확인했으므로, 넷스ㅔ이프 내비게이터처럼 모질라 브라우저로 인식.
    * 당시 통용되는 브라우저 식별 표기법을 깨트리는 것, 논란을 일으킴
    * 게다가 브라우저의 실제 버전은 문자열 중간에 숨겨짐
    * 모질라 3.0이 아니라 2.0으로 쓴 이유는 아직도 미스테리
    
#### 넷스케이프 커뮤니케이터 4와 IE 4~8
* 1997년 8월 넷스케이프 커뮤니케이터 4(내비게이터에서 커뮤니케이터로 바뀜) 출시
    * 사용자 에이전트 문자열 형식은 버전3과 동일하게 유지
* 마이크로소프트 IE4
    * 형식
        * Mozilla/4.0 (compatible; MSIE Version; Operation System)
    * 윈도 98에서 실행중인 IE4
        * Mozilla/4.0 (compatible; MSIE 4.0; Windows 98)
    * IE7까지 같은 패턴 사용
    * IE8에서 트라이던트라는 토큰 추가, 렌더링 엔진의 이름
    * 형식
        * Mozilla/4.0 (compatible; MSIE Version; Operation System; Trident/TridentVersion)
    * 예
        * Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)
    * 트라이던트 토큰은 개발자들이 IE8이 호환성 모드로 동작 중인지 쉽게 확인하도록 추가
    * IE9, 모질라 버전, 트라이던트 버전 5.0으로 변경
    * IE9 기본 사용자 에이전트 문자열
        * Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)
        
#### 게코
* 게코 렌더링 엔진은 파이어폭스의 핵심
* 게코가 처음 개발되었을 당시는 이후 넷스케이프6이 된 범용 모질라 브라우저의 일부분
* 넷스케이프6, 기존의 단순한 사용자 에이전트 문자열과 큰 차이가 생김
* 형식
    * Mozilla/MozillaVersion (Platform; Encryption; OS-or-CPU; Language; PrereleaseVersion)Gecko/GeckoVersion ApplicationProduct/ApplicationProductVersion
    
    ![9-1.png](img/9-1.png)
    
* 윈도 XP의 넷스케이프 6.21
    * Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:0.9.4) Gecko/20011128 Netscape6/6.2.1
* 리눅스의 SeaMonkey 1.1
    * Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.1b2) Gecko/20050823 SeaMonkey/1.1a
* 브라우저 버전이 각각 다르지만 게코 기반임을 알 수 있음
* 파이어폭스 4, 사용자 에이전트 문자열 단순화
    * 언어 토큰('en-US') 삭제
    * 암호화 토큰 삭제 'U'
    * 윈도에서 플랫폼 토큰 삭제
    * GeckoVersion 토큰은 'Gecko/20100101'로 고정
    * 파이어폭스 4 사용자 에이전트 문자열
        * Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox 4.0.1
        
#### 웹킷
* 애플은 2003년 사파리 웹 브라우저 발표
* 사파리의 렌더링엔진은 웹킷
* 웹킷은 리눅스 기반 웹 브라우저인 컨커러의 렌더링 엔진 KHTML 에서 분기
* 몇 년후 독자적인 오픈소스 프로젝트
* IE3 개발자들이 마주쳤던 문제
    * 인기있는 사이트가 이 브라우저에서 정확히 표시하게 되려면 어떻게 할것인가?
* 이 브라우저를 인기있는 브라우저와 호환된다고 판단할 정보를 사용자 에이전트 문자열에 제공
* 형식
    * Mozilla/5.0 (Platform; encryption; OS-or-CPU; anguage) AppleWebkit/AppleWebKitVersion (KHTML, like Gecko) Safari/SafariVersion
* 예제
    * Mozilla/5.0 (Macintosh; U; PPC Mac OS X; en) AppleWebKit/124 (KHTML, like Gecko) Safari/125.1
* 가장 흥미롭고 논란거리인 부분
    * (KHTML, like Gecko)
* 애플의 응답
    * 사파리는 모질라와 호환되며, 사파리 사용자가 비호환 브라우저로 잘못 간주되어 웹사이트로부터 차단 당해서는 안된다.
* 버전3, 사용자 에이저느 문자열은 조금 더 확장
* 현재 사용되는 사용자 에이전트 문자열
    * Mozilla/5.0 (Macintosh; U; PPC Mac OS X; en) AppleWebKit/522.15.5 (KHTML, like Gecko) Version/3.0.3 Safari/522.15.5

#### 컨커러
* KDE 리눅스 데스크톱 환경의 컨커러 브라우저
* 오픈 소스 렌더링 엔진인 KHTML을 렌더링 엔진으로 사용
* 호환성을 위해 사용자 에이전트 문자열 형식은 IE 비슷하게 만들기로 결정
* 형식
    * Mozilla/5.0 (compatible; Konqueror/Version; OS-or-CPU)
* 컨커러 3.2에서는 웹킷 사용자 에이전트 문자열을 따라 자신을 KHTML로 표시
* 형식
    * Mozilla/5.0 (compatible; Konqueror/Version; OS-or-CPU) KHTML/KHTMLVersion (like Gecko)
    
#### 크롬
* 구글의 크롬 브라우저는 렌더링 엔진으로 웹킷을 사용
* 자바스크립트엔진은 독자적으로 개발
* 사용자 에이전트 문자열에 웹킷에 들어가는 모든 정보와 크롬 정보도 포함
* 형식
    * Mozilla/5.0 (Platform; Encryption; OS-or-CPU; Language) AppleWebKit/AppleWebKitVersoin (KHTML, like Gecko) Chrome/ChromeVersion Safari/SafariVersion

#### 오페라
* 가장 논란이 많은 브라우저
* 버전 8 이전의 오페라, 형식
    * Opera/Version (OS-or-CPU; Encryption) [Language]
* 윈도 XP, 오페라 7.54의 사용자 에이전트 문자열
    * Opera/7.54 (Windows NT 5.1; U) [en]
* 오페라 8, 언어 부분을 괄호 안으로 옮겨서 다른 브라우저의 사용자 에이전트 문자열과 비슷하게 만듦
    * Opera/Version (OS-or-CPU; Encryption; Language)
* 윈도 XP, 오페라 8의 사용자 에이전트 문자열
    * Opera/8.0 (Windows NT 5.1; U; en)
* 현재 주요 브라우저 중에서 제품 이름과 버전을 완전히 정확히 표시하는 브라우저는 오페라가 유일
* 하지만 오페라 역시 다른 브라우저와 마찬가지로 자신만의 사용자 에이전트 문자열을 고집할 수 없었음
* 인터넷의 수많은 브라우저 탐지 코드에서 제품이름을 모질라라고 표시했을 것이라 가정
* 결국 오페라도 사용자 에이전트 문자열을 완전히 바꿔서 다른 브라우저처럼 보이게 함
* 오페라 9, 사용자 에이전트 문자열을 두 가지로 표시
    * 파이어폭스나 IE처럼 표시, 마지막에 opera 문자열과 버전 번호 추가
        * Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50
        * Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50
    * 파이어폭스나 IE로 가장하는 방법, Opera 문자열이나 버전정보도 표시하지 않음
        * 오페라를 구별 할 방법은 전혀 없음
        * 오페라가 사이트별로 다르게 설정
* 오페라 10, 사용자 에이전트 문자열 변경
    * Opera/9.80 (OS-or-CPU; Encryption; Language) Presto/PrestoVersion Version/Version
    * 오페라 9.8이란 버전은 존재하지 않지만, 오페라 개발자들은 부주의하게 개발된 브라우저 탐지 스크립트에서 Opera/10.0 을 오페라1로 잘못 판단 할 까봐 이렇게 만들었음
* 윈도 7, 오페라 10.63의 사용자 에이전트 문자열
    * Opera/9.80 (windows NT 6.1; U; en) Presto/2.6.30 Version/10.63
    
#### iOS와 안드로이드
* iOS와 안드로이드 기본 웹 브라우저는 웹킷 사용, 데스크탑과 같은 기본 사용자 에이전트 문자열 형식 공유
* iOS장치의 기본 형식
    * Mozilla/5.0 (Platform; Encryption; OS-or-CPU like Mac OS X; Language) AppleWebKit/AppleWebKitVersion (KHTML, like Gecko) Versoin/BrowserVersion Mobile/MoblieVersion Safari/SafariVersoin
    * 'like Mac OS X'를 추가하여 맥 운영체제를 식별하기 쉽게 함
    * Moblie 토큰 추가
    * Platform 은 장치에 따라 'iPhone', 'iPod', 'iPad'가 들어감
* 안드로이드 기본브라우저는 일반적으로 iOS 형식과 비슷하지만 mobile 토큰이 없음

### 9.3.2 브라우저 탐지 사용
* 특정 브라우저인지 판단하는 것은 대단히 복잡한 문제
* 특정 브라우저 정보가 꼭 필요한지부터 판단
* 일반적으로 렌더링 엔진이 무엇인지 알아내고 원하는 기능을 구현한 최소 버전이 무엇인지만 알아도 충분히 정확한 정보를 얻을 수 있다.

``` javascript
if (isIE6 || isIE7 ) {  // 이렇게 하지 마십시오
    // 코드
}
```

* 이 예제는 IE6,7일때만 코드 실행
* 이런 코드는 특정 브라우저 버전에 의존하므로 대단히 취약함
* 버전 8이 나오면 어떻게 해야 할까?
* 이 코드는 IE 새 버전이 나올 때마다 업데이트 해야 한다

```javascript
if (ieVer >= 6) {
    // 코드
}
```

#### 렌더링 엔진 식별
* 브라우저 이름과 버전은 렌더링 엔진만큼 중요하지는 않다.
* 파이어폭스, 넷스케이프, 카미노가 모두 같은 버전의 게코를 사용한다면 이들의 기능은 같다.
* 마찬가지로 사파리 3과 같은 버전의 웹킷을 사용하는 브라우저는 모두 같은 기능을 제공한다.
* 다음 스크립트는 다섯가지 주요 렌더링 엔진인 IE, 게코, 웹킷, KHTML, 오페라를 구별하는데 촛점을 맞춤

```javascript
// 기본 코드 구조
var client = function() {
    var engine = {
        // 렌더링 엔진
        ie: 0,
        gecko: 0,
        webkit: 0,
        khtml: 0,
        opera: 0,
        
        // 버전 문자열
        ver: null
    };
    
    // 렌더링 엔진과 플랫폼, 장치 탐지
    
    return {
        engine: engine
    };
}();
```

* 각 엔진을 프로퍼티로 나타내고 기본값을 0으로 설정
* 엔진을 찾으면 그 버전이 해당 프로퍼티에 부동소수점 숫자 값으로 저장
* 렌더링 엔진의 완전한 버전(문자열)은 ver 프로퍼티에 저장

```javascript
if (client.engine.ie) { // 브라우저가 IE라면 clien.engine.ie 값은 0보다 크다
    // IE 전용코드
} else if (client.engine.gecko > 1.5) {
    if (client.engine.ver == '1.8.1') {        
        // 해당 버전 전용코드
    }
}
````

* 렌더링 엔진을 정확히 식별하려면 정확한 순서로 테스트해야한다.
* 브라우저의 비일관성 때문에 테스트 순서가 틀리면 부정확한 결과를 얻게 됨
* 첫번째로 오페라인지 판단
    * 오페라의 사용자 에이전트 문자열이 다른 브라우저와 완전히 똑같을 수 있기 때문
    * 오페라의 사용자 에이전트 문자열은 자신을 오페라라고 밝히는 일이 없으므로 신뢰해선 안됨
* 오페라에서 실행 중인지 판단하려면 window.opera 객체가 있는지 확인

```javascript
if (window.opera) {
    engine.ver = window.opera.version();
    engine.opera = parseFloat(engine.ver);
}
```

* 논리적인 다음 단계는 웹킷인지 확인
    * 웹킷의 사용자 에이전트 문자열에는 'Gecko'와 'khtml'이 모두 들어가므로 다른 렌더링 엔진을 먼저 확인한다면 부정확한 결과를 얻을 것이다.
    * 문자열 'AppleWebKit'이 포함되는 사용자 에이전트 문자열은 웹킷이 유일

In [ ]:
var ua = navitator.userAgent;

if (window.opera) {
    engine.ver = window.opera.version();
    engine.opera = parseFloat(engine.ver);
} else if (/AppleWebKit\/(\S+)/.test(ua)){
    engine.ver = RegExp['$1'];
    engine.webkit = parseFloat(engine.ver);
}

* 다음 테스트할 엔진은 KHTML
    * KHTML 의 사용자 에이전트 문자열에도 'Gecko'가 있으므로 먼저 배제해야함

In [ ]:
var ua = navitator.userAgent;

if (window.opera) {
    engine.ver = window.opera.version();
    engine.opera = parseFloat(engine.ver);
} else if (/AppleWebKit\/(\S+)/.test(ua)) {
    engine.ver = RegExp['$1'];
    engine.webkit = parseFloat(engine.ver);
} else if (/KHTML\/(\S+)/.test(ua) || /Konqueror\/([^;]+)/.test(ua)){
    engine.ver = RegExp["$1"];
    engine.khtml = parseFloat(engine.ver);
}

* 웹킷과 KHTML을 모두 배제한 뒤에 Gecko 체크
* 실제 게코버전은 rv: 다음에 나타나기 때문에 이전 보다 더 복잡한 정규표현식 사용

In [ ]:
var ua = navitator.userAgent;

if (window.opera) {
    engine.ver = window.opera.version();
    engine.opera = parseFloat(engine.ver);
} else if (/AppleWebKit\/(\S+)/.test(ua)) {
    engine.ver = RegExp['$1'];
    engine.webkit = parseFloat(engine.ver);
} else if (/KHTML\/(\S+)/.test(ua) || /Konqueror\/([^;]+)/.test(ua)){
    engine.ver = RegExp["$1"];
    engine.khtml = parseFloat(engine.ver);
} else if (/rv:([^\)]+)\) Gecko\/\d{8}/.test(ua)){    
    engine.ver = RegExp["$1"];
    engine.gecko = parseFloat(engine.ver);
}

* 마지막 렌더링 엔진은 IE
* 버전번호는 MSIE 와 ; 사이

In [ ]:
var ua = navitator.userAgent;

if (window.opera) {
    engine.ver = window.opera.version();
    engine.opera = parseFloat(engine.ver);
} else if (/AppleWebKit\/(\S+)/.test(ua)) {
    engine.ver = RegExp['$1'];
    engine.webkit = parseFloat(engine.ver);
} else if (/KHTML\/(\S+)/.test(ua) || /Konqueror\/([^;]+)/.test(ua)){
    engine.ver = RegExp["$1"];
    engine.khtml = parseFloat(engine.ver);
} else if (/rv:([^\)]+)\) Gecko\/\d{8}/.test(ua)){    
    engine.ver = RegExp["$1"];
    engine.gecko = parseFloat(engine.ver);
} else if (/MSIE ([^;]+)/.test(ua)){    
    engine.ver = RegExp["$1"];
    engine.ie = parseFloat(engine.ver);
}

#### 브라우저 확인
* 대개는 브라우저의 렌더링 엔진만 알아내도 충분히 대응 할 수 있음
* 하지만 렌더링 엔진만으로는 자바스크립트의 기능에 대해서 알 수 없을 때가 있다.
* 애플의 사파리 브라우저와 구글의 크롬브라우저는 모두 웹킷 렌더링 엔진을 사용하지만 자바스크립트 엔진은 서로 다르다.
* 두 브라우저는 모두 client.engine.webkit 을 반환하지만 이것만으로는 충분히 명시적이지 않을 수 있다.
* 다음 예제 처럼 새로운 프로퍼티를 추가하면 도움이 된다.

```javascript
var client = function(){

    //rendering engines
    var engine = {            
        ie: 0,
        gecko: 0,
        webkit: 0,
        khtml: 0,
        opera: 0,

        //complete version
        ver: null  
    };
    
    //browsers
    var browser = {
        
        //browsers
        ie: 0,
        firefox: 0,
        safari: 0,
        konq: 0,
        opera: 0,
        chrome: 0,

        //specific version
        ver: null
    };
    
    // 렌더링 엔진, 플랫폼, 장치 탐지
    
    //return it
    return {
        engine:     engine,
        browser:    browser,      
    };
}();
```

* 각 주요 브라우저의 프로퍼티를 담을 browser 변수 추가

In [ ]:
// 브라우저와 렌더링 엔진 탐지
var ua = navitator.userAgent;

if (window.opera) {
    engine.ver = browser.ver = window.opera.version();
    engine.opera = browser.opera = parseFloat(engine.ver);
} else if (/AppleWebKit\/(\S+)/.test(ua)) {
    engine.ver = RegExp['$1'];
    engine.webkit = parseFloat(engine.ver);
    
    // 크롬인지 사파리 인지 판단
    if (/Chrome\/(\S+)/.test(ua)){
        browser.ver = RegExp["$1"];
        browser.chrome = parseFloat(browser.ver);
    } else if (/Version\/(\S+)/.test(ua)){
        browser.ver = RegExp["$1"];
        browser.safari = parseFloat(browser.ver);
    } else {
        //approximate version
        var safariVersion = 1;
        if (engine.webkit < 100){
            safariVersion = 1;
        } else if (engine.webkit < 312){
            safariVersion = 1.2;
        } else if (engine.webkit < 412){
            safariVersion = 1.3;
        } else {
            safariVersion = 2;
        }   

        browser.safari = browser.ver = safariVersion;        
    }
} else if (/KHTML\/(\S+)/.test(ua) || /Konqueror\/([^;]+)/.test(ua)){
    engine.ver = browser.ver = RegExp["$1"];
    engine.khtml = browser.konq = parseFloat(engine.ver);
} else if (/rv:([^\)]+)\) Gecko\/\d{8}/.test(ua)){    
    engine.ver = RegExp["$1"];
    engine.gecko = parseFloat(engine.ver);
    
    // 파이어폭스인지 판단
    if (/Firefox\/(\S+)/.test(ua)){
        browser.ver = RegExp["$1"];
        browser.firefox = parseFloat(browser.ver);
    }
} else if (/MSIE ([^;]+)/.test(ua)){    
    engine.ver = browser.ver = RegExp["$1"];
    engine.ie = browser.ie = parseFloat(engine.ver);
}

* 오페라와 IE의 경우 browser객체에 저장된 값은 engine 객체에 저장된 값과 같다
* 컨커러에서는 browser.konq과 engine.khtml이 일치, browser.ver 와 engine.ver 이 일치
* 크롬 버전, Chrome 문자열을 찾고 그 뒤의 숫자를 가져옴
* 사파리 버전, Version 문자열 다음의 숫자
* 파이어폭스, Firefox 문자열 다음의 숫자

#### 플랫폼 감지
* 대개는 렌더링 엔진만 알면 충분하다
* 하지만 플랫폼을 알아야 할 상황이 있다.
* 사파리나 파이어폭스, 오페라 등 여러 플랫폼에서 동작하도록 만들어진 브라우저는 플랫폼 별로 다른 문제가 있을 수 있다.
* 주요 플랫폼은 윈도, 맥, 유닉스(리눅스 포함)이다.
* 플랫폼 탐지를 위한 새로운 변수를 추가

```javascript
var client = function(){

    //rendering engines
    var engine = {            
        ie: 0,
        gecko: 0,
        webkit: 0,
        khtml: 0,
        opera: 0,

        //complete version
        ver: null  
    };
    
    //browsers
    var browser = {
        
        //browsers
        ie: 0,
        firefox: 0,
        safari: 0,
        konq: 0,
        opera: 0,
        chrome: 0,

        //specific version
        ver: null
    };
    
    var system = {
        win: false,
        mac: false,
        x11: false
    };
    
    // 렌더링 엔진, 플랫폼, 장치 탐지
    
    //return it
    return {
        engine:     engine,
        browser:    browser,      
    };
}();
```
* x11 은 유닉스
* 렌더링 엔진과 달리 플랫폼 정보는 매우 제한적이며 운영체제나 버전 정보를 얻기는 쉽지 않다.
* 새 플랫폼중 운영체제 버전을 제공하는 플랫폼은 윈도가 유일
* 숫자대신 불리언 할당
* 플랫폼을 확인 할 때는 navigator.platform 확인

```javascript
var p = navigator.platform;
system.win = p.indexOf("Win") == 0;
system.mac = p.indexOf("Mac") == 0;
system.x11 = (p == "X11") || (p.indexOf("Linux") == 0);
```

#### 윈도 운영체제 식별
* 플랫폼이 윈도일 경우, 사용자 에이전트 문자열에서 운영체제 정보를 얻을 수 있다.
![9-2.png](img/9-2.png)

```javascript
if (system.win){
    if (/Win(?:dows )?([^do]{2})\s?(\d+\.\d+)?/.test(ua)){
        if (RegExp["$1"] == "NT"){
            switch(RegExp["$2"]){
                case "5.0":
                    system.win = "2000";
                    break;
                case "5.1":
                    system.win = "XP";
                    break;
                case "6.0":
                    system.win = "Vista";
                    break;
                case "6.1":
                    system.win = "7";
                    break;
                default:
                    system.win = "NT";
                    break;                
            }                            
        } else if (RegExp["$1"] == "9x"){
            system.win = "ME";
        } else {
            system.win = RegExp["$1"];
        }
    }
}
```

#### 모바일 장치 식별
* 모바일 장치가 해당하는 프로퍼티 추가

```javascript
var client = function(){

    //rendering engines
    var engine = {            
        ie: 0,
        gecko: 0,
        webkit: 0,
        khtml: 0,
        opera: 0,

        //complete version
        ver: null  
    };
    
    //browsers
    var browser = {
        
        //browsers
        ie: 0,
        firefox: 0,
        safari: 0,
        konq: 0,
        opera: 0,
        chrome: 0,

        //specific version
        ver: null
    };
    
    var system = {
        win: false,
        mac: false,
        x11: false,
        
        // 모바일 장치
        iphone: false,
        ipod: false,
        ipad: false,
        ios: false,
        android: false,
        nokiaN: false,
        winMobile: false
    };
    
    // 렌더링 엔진, 플랫폼, 장치 탐지
    
    //return it
    return {
        engine:     engine,
        browser:    browser,      
    };
}();
```
* ios 장치는 문자열 'iphone', 'ipod', 'ipad'만 검색 하면 됨

In [ ]:
system.iphone = ua.indexOf("iPhone") > -1;
system.ipod = ua.indexOf("iPod") > -1;
system.ipad = ua.indexOf("iPad") > -1;

// IOS 버전 판단
if (system.mac && ua.indexOf("Mobile") > -1){
    if (/CPU (?:iPhone )?OS (\d+_\d+)/.test(ua)){
        system.ios = parseFloat(RegExp.$1.replace("_", "."));
    } else {
        system.ios = 2;  //can't really detect - so guess
    }
}

* 안드로이드 운영체제는 문자열 'Android'를 검색하기만 하면 되고 버전은 바로 뒤에 있음

In [ ]:
// 안드로이드 버전 판단
if (/Android (\d+\.\d+)/.test(ua)){
    system.android = parseFloat(RegExp.$1);
}

* 노키아 스마트폰
    * 사용자 에이전트 문자열에 'safari' 가 있지만 실제 사파리는 아님
    * 문자열 'NokiaN'만 체크 하면 됨
    
```javascript
system.nokiaN = ua.indexOf("NokiaN") > -1;
```

* 그 외, 윈도우 모바일 및 게임 시스템 등등이 있음

In [ ]:
// 전체 코드
var client = function(){

    // 렌더링 엔진
    var engine = {            
        ie: 0,
        gecko: 0,
        webkit: 0,
        khtml: 0,
        opera: 0,

        // 버전 문자열
        ver: null  
    };
    
    // 브라우저
    var browser = {
        
        // 브라우저
        ie: 0,
        firefox: 0,
        safari: 0,
        konq: 0,
        opera: 0,
        chrome: 0,

        // 버전 문자열
        ver: null
    };

    
    // 플랫폼, 장치, 운영체제
    var system = {
        win: false,
        mac: false,
        x11: false,
        
        // 모바일 장치
        iphone: false,
        ipod: false,
        ipad: false,
        ios: false,
        android: false,
        nokiaN: false,
        winMobile: false,
        
        // 게임용 장치
        wii: false,
        ps: false 
    };    

    // 렌더링 엔진 및 브라우저 탐지
    var ua = navigator.userAgent;    
    if (window.opera){
        engine.ver = browser.ver = window.opera.version();
        engine.opera = browser.opera = parseFloat(engine.ver);
    } else if (/AppleWebKit\/(\S+)/.test(ua)){
        engine.ver = RegExp["$1"];
        engine.webkit = parseFloat(engine.ver);
        
        // 크롬인지 사파리인지 판단
        if (/Chrome\/(\S+)/.test(ua)){
            browser.ver = RegExp["$1"];
            browser.chrome = parseFloat(browser.ver);
        } else if (/Version\/(\S+)/.test(ua)){
            browser.ver = RegExp["$1"];
            browser.safari = parseFloat(browser.ver);
        } else {
            // 비슷한 버전
            var safariVersion = 1;
            if (engine.webkit < 100){
                safariVersion = 1;
            } else if (engine.webkit < 312){
                safariVersion = 1.2;
            } else if (engine.webkit < 412){
                safariVersion = 1.3;
            } else {
                safariVersion = 2;
            }   
            
            browser.safari = browser.ver = safariVersion;        
        }
    } else if (/KHTML\/(\S+)/.test(ua) || /Konqueror\/([^;]+)/.test(ua)){
        engine.ver = browser.ver = RegExp["$1"];
        engine.khtml = browser.konq = parseFloat(engine.ver);
    } else if (/rv:([^\)]+)\) Gecko\/\d{8}/.test(ua)){    
        engine.ver = RegExp["$1"];
        engine.gecko = parseFloat(engine.ver);
        
        // 파이어폭스인지 판단
        if (/Firefox\/(\S+)/.test(ua)){
            browser.ver = RegExp["$1"];
            browser.firefox = parseFloat(browser.ver);
        }
    } else if (/MSIE ([^;]+)/.test(ua)){    
        engine.ver = browser.ver = RegExp["$1"];
        engine.ie = browser.ie = parseFloat(engine.ver);
    }
    
    // 브라우저 탐지
    browser.ie = engine.ie;
    browser.opera = engine.opera;
    

    // 플랫폼 탐지
    var p = navigator.platform;
    system.win = p.indexOf("Win") == 0;
    system.mac = p.indexOf("Mac") == 0;
    system.x11 = (p == "X11") || (p.indexOf("Linux") == 0);

    // 윈도 운영체제 탐지
    if (system.win){
        if (/Win(?:dows )?([^do]{2})\s?(\d+\.\d+)?/.test(ua)){
            if (RegExp["$1"] == "NT"){
                switch(RegExp["$2"]){
                    case "5.0":
                        system.win = "2000";
                        break;
                    case "5.1":
                        system.win = "XP";
                        break;
                    case "6.0":
                        system.win = "Vista";
                        break;
                    case "6.1":
                        system.win = "7";
                        break;
                    default:
                        system.win = "NT";
                        break;                
                }                            
            } else if (RegExp["$1"] == "9x"){
                system.win = "ME";
            } else {
                system.win = RegExp["$1"];
            }
        }
    }
    
    // 모바일 장치
    system.iphone = ua.indexOf("iPhone") > -1;
    system.ipod = ua.indexOf("iPod") > -1;
    system.ipad = ua.indexOf("iPad") > -1;
    system.nokiaN = ua.indexOf("NokiaN") > -1;
    
    // 윈도 모바일
    if (system.win == "CE"){
        system.winMobile = system.win;
    } else if (system.win == "Ph"){
        if(/Windows Phone OS (\d+.\d+)/.test(ua)){;
            system.win = "Phone";
            system.winMobile = parseFloat(RegExp["$1"]);
        }
    }
    
    
    // iOS 버전 판단
    if (system.mac && ua.indexOf("Mobile") > -1){
        if (/CPU (?:iPhone )?OS (\d+_\d+)/.test(ua)){
            system.ios = parseFloat(RegExp.$1.replace("_", "."));
        } else {
            system.ios = 2;  // 정확히 판단할 수 없으므로 짐작한 버전
        }
    }
    
    // 안드로이드 버전 판단
    if (/Android (\d+\.\d+)/.test(ua)){
        system.android = parseFloat(RegExp.$1);
    }
    
    // 게임 시스템
    system.wii = ua.indexOf("Wii") > -1;
    system.ps = /playstation/i.test(ua);
    
    // 결과 반환
    return {
        engine:     engine,
        browser:    browser,
        system:     system        
    };

}();

### 9.3.4 사용법
* 클라이언트 탐지의 마지막 옵션으로 생각해야 한다.
* 가능하다면 기능탐지나 쿽스탐지를 먼저 시도해야 한다.
* 다음과 같은 상황에서 알맞다.
    * 기능이나 쿽스를 직접 정확히 탐지 할 수 없을 때
    * 같은 브라우저의 기능이 플랫폼 별로 다를 때
    * 정보 수집 목적으로 정확히 어떤 브라우저 인지 알아야 할 때

#### 참고
* 맥북
    * 크롬
        * Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36
    * 사파리
        * Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15
    * 파이어폭스
        * Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:73.0) Gecko/20100101 Firefox/73.0
* 윈도우 10
    * 크롬
        * Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36
    * IE 11
        * Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E; .NET CLR 2.0.50727; .NET CLR 3.0.30729; .NET CLR 3.5.30729)
    * IE 엣지
        * Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18362